In [4]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator 

In [6]:
filename='BTC-USD.csv'
df=pd.read_csv(filename)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       367 non-null    object 
 1   Open       367 non-null    float64
 2   High       367 non-null    float64
 3   Low        367 non-null    float64
 4   Close      367 non-null    float64
 5   Adj Close  367 non-null    float64
 6   Volume     367 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 20.2+ KB


In [7]:
df['Date']=pd.to_datetime(df['Date'])


In [8]:
df.set_axis(df['Date'],inplace=True)
#index set as date

In [9]:
df.drop(columns=['Open','High','Low','Volume'],inplace=True)
df.head()

,Date,Close,Adj Close
Date,,,
2019-12-18,2019-12-18,7276.802734,7276.802734
2019-12-19,2019-12-19,7202.844238,7202.844238
2019-12-20,2019-12-20,7218.816406,7218.816406
2019-12-21,2019-12-21,7191.158691,7191.158691
2019-12-22,2019-12-22,7511.588867,7511.588867


In [10]:
!pip install plotly==3.6.0
import plotly
import plotly.graph_objs as go
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
print("Plotly Version: ",plotly.__version__)

init_notebook_mode(connected=True)
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

     |████████████████████████████████| 31.1MB 153kB/s 
  Created wheel for plotly: filename=plotly-3.6.0-py2.py3-none-any.whl size=38330571 sha256=a464dcf65fb108f68694ec2eb977f7de7595b3a1a994ab0cb7847667a8b8e2a5
  Stored in directory: /root/.cache/pip/wheels/67/0b/29/08c7f5caed2d1ac446db982ff607b326d49bfa0bd3a67ef8c7
Successfully built plotly
ERROR: cufflinks 0.17.3 has requirement plotly>=4.1.1, but you'll have plotly 3.6.0 which is incompatible.
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
Plotly Version:  3.6.0


In [11]:
configure_plotly_browser_state()

In [12]:
configure_plotly_browser_state()
trace=go.Scatter(x=df['Date'],
                 y=df['Close'],
                 mode='lines',
                 name='Data'
                  )
layout=go.Layout(title="",
                xaxis={'title' : "Date"},
                yaxis={'title':"Close(Dollars)"}
                )
fig=go.Figure(data=[trace],
             layout=layout
             )
iplot(fig)

In [13]:
close_data=df['Close'].values
close_data=close_data.reshape((-1,1))

split_percent=0.80
split=int(split_percent*len(close_data))

close_train=close_data[:split]
close_test=close_data[split:]


data_train=df['Date'][:split]
data_test=df['Date'][split:]

print(len(close_train))
print(len(close_test))



293
74


In [14]:
look_back=15
train_generator=TimeseriesGenerator(close_train,
                                   close_train,
                                    length=look_back,
                                    batch_size=20
                                   )
test_generator=TimeseriesGenerator(close_test,
                                   close_test,
                                    length=look_back,
                                    batch_size=1
                                   )

                                   
                                    

# neural network

In [15]:
from keras.models import Sequential
from keras.layers import LSTM,Dense

model=Sequential()
model.add(
LSTM(10,
     activation='relu',
     input_shape=(look_back,1)
    ))
model.add(Dense(1))
model.compile(optimizer='adam',loss='mse')

In [16]:
model.fit_generator(train_generator,epochs=100, verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



Epoch 1/100
14/14 [==============================] - 1s 6ms/step - loss: 30177454.4000
Epoch 2/100
14/14 [==============================] - 0s 6ms/step - loss: 24339491.6000
Epoch 3/100
14/14 [==============================] - 0s 5ms/step - loss: 27073539.6000
Epoch 4/100
14/14 [==============================] - 0s 6ms/step - loss: 12807628.4667
Epoch 5/100
14/14 [==============================] - 0s 6ms/step - loss: 8154339.1000
Epoch 6/100
14/14 [==============================] - 0s 6ms/step - loss: 8499825.6000
Epoch 7/100
14/14 [==============================] - 0s 6ms/step - loss: 5794592.4000
Epoch 8/100
14/14 [==============================] - 0s 6ms/step - loss: 4156195.0500
Epoch 9/100
14/14 [==============================] - 0s 5ms/step - loss: 5319808.1417
Epoch 10/100
14/14 [==============================] - 0s 5ms/step - loss: 10902317.9000
Epoch 11/100
14/14 [==============================] - 0s 5ms/step - loss: 47041886.9333
Epoch 12/100
14/14 [==========================

In [17]:
prediction=model.predict_generator(test_generator)
prediction

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning:

`Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.



array([[11512.787],
       [11694.537],
       [11836.182],
       [11917.902],
       [11787.406],
       [11986.968],
       [12185.039],
       [12236.218],
       [12205.106],
       [12307.795],
       [12416.765],
       [12744.284],
       [13140.704],
       [13238.749],
       [13549.756],
       [13605.204],
       [13803.02 ],
       [13939.224],
       [13851.39 ],
       [14074.374],
       [14187.81 ],
       [14491.451],
       [14586.87 ],
       [14527.181],
       [14741.297],
       [14739.851],
       [15100.157],
       [19128.469],
       [15967.055],
       [15777.68 ],
       [16126.984],
       [16120.072],
       [16476.797],
       [16774.863],
       [16970.365],
       [17177.008],
       [17347.592],
       [17274.629],
       [17485.525],
       [17876.791],
       [18422.604],
       [18828.572],
       [18604.4  ],
       [18759.98 ],
       [19017.516],
       [18824.299],
       [18617.291],
       [18494.146],
       [18860.734],
       [18571.273],


In [18]:
close_train=close_train.reshape((-1))
close_test=close_test.reshape((-1))
prediction=prediction.reshape((-1))

In [19]:
configure_plotly_browser_state()

trace1=go.Scatter(x=data_train,
                  y=close_train,
                  mode='lines',
                  name='data'
                  
                 )
trace2=go.Scatter(x=data_test,
                 y=prediction,
                  mode='lines',
                  name='prediction'
                  
                 )
trace3=go.Scatter(x=data_test,
                 y=close_test,
                 mode='lines',
                 name ='Ground Truth')
layout=go.Layout(title='BTC-USD',
                xaxis={'title':"Date"},
                yaxis={'title':"Close"},
                 
                )
fig=go.Figure(data=[trace1,trace2,trace3],layout=layout)
iplot(fig)